In [1]:
%load_ext autoreload
%autoreload 2
import os
import matplotlib.pyplot as plt
import seaborn as sns
from os.path import join
from tqdm import tqdm
import joblib
import sasc.config
from sasc import config
import numpy as np
import sasc.viz
from sasc import analyze_helper
from sasc.config import FMRI_DIR, RESULTS_DIR
import dvu
import pandas as pd
dvu.set_style()

### Load pilot pickle

In [2]:
# pilot_name = 'pilot_story_data.pkl'
# pilot_name = 'pilot3_story_data.pkl'
pilot_name = 'pilot4_story_data.pkl'
# pilot_name = 'pilot5_story_data.pkl'
stories_data_dict = joblib.load(
    join(config.RESULTS_DIR, 'processed', pilot_name))
if pilot_name == 'pilot_story_data.pkl':
    pilot_data_dir = join(config.PILOT_STORY_DATA_DIR, '20230504')
elif pilot_name == 'pilot3_story_data.pkl':
    pilot_data_dir = join(config.PILOT_STORY_DATA_DIR, '20231106')
elif pilot_name == 'pilot4_story_data.pkl':
    pilot_data_dir = join(config.PILOT_STORY_DATA_DIR, '20240509')
elif pilot_name == 'pilot5_story_data.pkl':
    pilot_data_dir = join(config.PILOT_STORY_DATA_DIR, '20240604')

# load responses
# story_idxs = np.where(
    # np.array(stories_data_dict['story_setting']) == 'default')[0]
story_idxs = np.arange(len(stories_data_dict['story_name_new']))
resp_np_files = [stories_data_dict['story_name_new'][i].replace('_resps', '')
                 for i in story_idxs]
resps_dict = {
    k: np.load(join(pilot_data_dir, k))
    for k in tqdm(resp_np_files)
}

100%|██████████| 4/4 [00:15<00:00,  3.87s/it]


### Let's check the alignment

In [3]:
timings_list = stories_data_dict['timing']
story_names_list = list(resps_dict.keys())
resps = list(resps_dict.values())

In [4]:
TRIM = 5
for i in range(len(resps)):
    t = timings_list[i]
    duration_secs = t['time_running'].max()
    print(story_names_list[i], 'resp_length',
          resps[i].shape[0],
          'story_text_trs', str(int(duration_secs // 2)) + '±1')  # , 'timings',
    diff = int(duration_secs // 2) - resps[i].shape[0]
    print(f'\t{(abs(diff - TRIM * 2))=}')

deeptune-story19.npy resp_length 372 story_text_trs 376±1
	(abs(diff - TRIM * 2))=6
deeptune-story20.npy resp_length 391 story_text_trs 395±1
	(abs(diff - TRIM * 2))=6
deeptune-story21.npy resp_length 438 story_text_trs 442±1
	(abs(diff - TRIM * 2))=6
deeptune-story22.npy resp_length 355 story_text_trs 359±1
	(abs(diff - TRIM * 2))=6


### Let's check the alignment w/ paragraphs

In [134]:
# for story_num in [0]:
for story_num in story_idxs:
    rows = stories_data_dict["rows"][story_num]

    # get resp_chunks
    resp_story = resps_dict[
        stories_data_dict["story_name_new"][story_num].replace(
            '_resps', '')
    ].T  # (voxels, time)
    timing = stories_data_dict["timing"][story_num]
    if 'paragraphs' in stories_data_dict.keys():
        paragraphs = stories_data_dict["paragraphs"][story_num]
    else:
        paragraphs = stories_data_dict["story_text"][story_num].split("\n\n")

    if pilot_name in ['pilot3_story_data.pkl']:
        paragraphs = [sasc.analyze_helper.remove_repeated_words(
            p) for p in paragraphs]
    assert len(paragraphs) == len(rows), f"{len(paragraphs)} != {len(rows)}"
    resp_chunks = analyze_helper.get_resps_for_paragraphs(
        timing, paragraphs, resp_story, offset=2)
    assert len(resp_chunks) <= len(
        paragraphs), f"{len(resp_chunks)} > {len(paragraphs)}"

In [3]:
start_indexes, end_indexes = analyze_helper.get_start_end_indexes_for_paragraphs(
    timing, paragraphs)
print(paragraphs[0].split())
print(timing['word'].values.tolist())
print(
    f'should have {len(paragraphs)} paragraphs but only found {len(resp_chunks)}')
for i in range(min(len(paragraphs), len(start_indexes))):
    print(i, '\t', len(paragraphs[i].split()),
          '\t', end_indexes[i] - start_indexes[i])
    # print(i, timing['word'].values.tolist()[start_indexes[i]:end_indexes[i]])
    # print(i, resp_chunks[i].shape)
# print([len(p.split()) for p in paragraphs])

NameError: name 'timing' is not defined

In [136]:
paragraphs[-1]

"As I stood there, lost in my thoughts about the various aspects of my life that had been influenced by age, locations, experiences, and relationships, I couldn't help but notice how quickly time seemed to pass. It felt like just yesterday when, weeks earlier, I had embarked on a new adventure that had challenged me to step out of my comfort zone and embrace the unknown. Weeks later, I found myself reflecting on the lessons I had learned and the memories I had created during that time. And just a few weeks later, it was time for me to embark on yet another journey, one that would undoubtedly bring new challenges and opportunities for growth. With each passing week, I became more aware of the precious nature of time and the importance of making the most of each moment. As I continued to grow older, I was determined to live a life filled with love, laughter, and meaningful experiences that would stand the test of time."

In [137]:
# note: final paragraph of story wasn't run
x = '\n'.join(paragraphs)
i = x.index('thick and thin,')
i_final_paragraph = x[i:].index('As the couple')
x[i + i_final_paragraph:]

ValueError: substring not found